AE para dashboard rh

In [305]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.transform import factor_cmap, cumsum
from bokeh.plotting import figure
from bokeh.palettes import Spectral6, Category20c
from math import pi

In [138]:
output_notebook()

Loading BokehJS ...

In [218]:
data = pd.read_excel('./data/db.xls')
data.head(5)

,Usuario,Nombre Completo,Fecha de Última Contratación,Nombre Departamento,Estado del Empleado,Título de la Posición,Vacante,Función,Tipo de Colaborador,Tipo de Contratación,Tipo de Nómina,Pertenece a Sindicato,Ubicación,Tipo de contrato,Tipo de jornada
0,U1174488,A,2016-07-18,Abastecimiento,Activo,Encargado de Grupo Limpieza General,False,Administrativo,Colaborador,Tiempo Completo,Planilla (Semanal),NaN,Guatemala,Tiempo Indefinido,Mixta
1,U1202277,B,2015-10-01,Manufactura,Con terminación de contrato,Empacador,False,Operarios,Colaborador,Tiempo Completo,Planilla (Semanal),NaN,Costa Rica,Tiempo Indefinido,Mixta
2,U1174527,C,2018-03-05,Servicio Técnico,Activo,Instructor Centro,False,Administrativo,Colaborador,Tiempo Completo,Nómina (Mensual),No,Guatemala,Tiempo Indefinido,Diurna
3,U1174644,D,2016-09-19,Abastecimiento,Activo,Conserje Limpieza de Silos y Patios,False,Operarios,Colaborador,Tiempo Completo,Planilla (Semanal),No,Guatemala,Tiempo Indefinido,Mixta
4,U1229722,E,2020-05-27,Manufactura,Activo,Operador - C,False,Operarios,Colaborador,Tiempo Completo,Quincenal,No,República Dominicana,Por Tiempo Indefinido,Ordinaria


## Numero de vacantes por ubicación

In [145]:
vpu = data.loc[data['Vacante'] == True]\
    .groupby('Ubicación')\
    .count()[['Usuario']]\
    .sort_values('Usuario', ascending=False)\
    .reset_index()
vpu

,Ubicación,Usuario
0,Guatemala,17812
1,El Salvador,11625
2,Costa Rica,2576
3,Honduras,956
4,República Dominicana,572
5,México,395
6,Nicaragua,51
7,Estados Unidos de América,37
8,Barbados,2
9,Panamá,1


In [158]:
source = ColumnDataSource(data=vpu.head(5))
p = figure(x_range=vpu['Ubicación'], plot_height=350, plot_width=900, toolbar_location=None, title="Vacantes por ubicación")
p.vbar(x='Ubicación', top='Usuario', width=0.9, source=source, legend_field="Ubicación",
       line_color='white')
show(p)

# Cantidad de empleados por jornada con terminación de contrato

In [125]:
data['Tipo de jornada'].unique() # need clean up

array(['  Mixta      ', 'Mixta        ', '  Diurna      ',
       ' Ordinaria       ', ' Mixta       ', '       Mixta ',
       '       Diurna ', '     Diurna   ', 'Diurna        ',
       '   Diurna     ', '   Nocturna     ', '  Nocturna      ',
       '   Mixta     ', '    Diurna    ', '    Mixta    ',
       '       Nocturna ', '   Rotativa     ', 'Nocturna        ',
       'Ordinario        ', '      Diurna  ', '        ',
       '        Diurna', 'Continua        ', '    Nocturna    ',
       'Rotativa        ', 'Acumulativa Semanal        ',
       '  Exentos      ', nan, 'Discontinua        ', '        No Aplica',
       ' Nocturna       ', ' Diurna       '], dtype=object)

In [126]:
 data['Tipo de jornada'] = data['Tipo de jornada'].apply(lambda x: x.strip() if type(x) == str else x)

In [127]:
 data['Tipo de jornada'] = data['Tipo de jornada'].apply(lambda x: 'Ordinaria' if x == 'Ordinario' else x)

In [160]:
ept = data.loc[data['Estado del Empleado'] == 'Con terminación de contrato']\
    [['Estado del Empleado', 'Tipo de jornada']]\
    .groupby('Tipo de jornada')\
    .count().reset_index()
ept

,Tipo de jornada,Estado del Empleado
0,,6
1,Acumulativa Semanal,1
2,Continua,5
3,Discontinua,1
4,Diurna,23186
5,Mixta,14148
6,No Aplica,1
7,Nocturna,795
8,Ordinaria,1076
9,Rotativa,70


In [162]:
source = ColumnDataSource(data=ept)
p = figure(x_range=ept['Tipo de jornada'], plot_height=350, plot_width=900, toolbar_location=None, title="Empleados por jornada con terminación de contrato")
p.vbar(x='Tipo de jornada', top='Estado del Empleado', width=0.9, source=source, legend_field="Tipo de jornada",
       line_color='white')
show(p)

# Usuarios por nomina

In [129]:
data.groupby('Tipo de Nómina')\
    .count()[['Usuario']]\
    .sort_values('Usuario', ascending=False)

,Usuario
Tipo de Nómina,
Nómina (Mensual),35766
Planilla General (Quincenal),14293
Quincenal,11638
Planilla (Semanal),3541
Planilla Ejecutiva (Mensual),77
Quincenal Confidencial,47
Servicios Profesionales,2


## Empleados que pertenecen a sindicato por ubicación

In [179]:
epsu = data.loc[data['Pertenece a Sindicato'] == 'Si']\
    .groupby(['Pertenece a Sindicato', 'Ubicación'])\
    .count()[['Usuario']]\
    .reset_index()\
    .sort_values('Usuario', ascending=False)
epsu

,Pertenece a Sindicato,Ubicación,Usuario
2,Si,Guatemala,25
0,Si,Costa Rica,16
5,Si,Nicaragua,15
1,Si,El Salvador,14
3,Si,Honduras,6
4,Si,México,3
6,Si,República Dominicana,2


In [201]:
source = ColumnDataSource(data=epsu)
p = figure(x_range=epsu['Ubicación'], plot_height=350, plot_width=600, title="Empleados por jornada con terminación de contrato")
p.vbar(x='Ubicación', top='Usuario', width=0.9, source=source, legend_field="Ubicación",
       line_color='white')
show(p)

## Contrataciones por año

In [189]:
data['Año Última Contratación'] = data['Fecha de Última Contratación'].apply(lambda x: x.year)

In [196]:
cp = data.groupby('Año Última Contratación')\
    .count()[['Usuario']]\
    .sort_values('Año Última Contratación')\
    .reset_index()
cp.head(5)

,Año Última Contratación,Usuario
0,1970,1
1,1973,1
2,1974,1
3,1975,3
4,1976,4


In [203]:
p = figure(title="Contrataciones por año", plot_height=350, plot_width=600, x_axis_label="Año", y_axis_label="Contrataciones")
p.line(cp['Año Última Contratación'], cp['Usuario'], line_width=2)
show(p)

In [225]:
data.groupby(['Tipo de Nómina', 'Ubicación'])\
    .count()\
    .reset_index()\
    [['Tipo de Nómina', 'Ubicación', 'Usuario']]

,Tipo de Nómina,Ubicación,Usuario
0,Nómina (Mensual),Barbados,9
1,Nómina (Mensual),Costa Rica,1892
2,Nómina (Mensual),El Salvador,502
3,Nómina (Mensual),Guatemala,28541
4,Nómina (Mensual),Honduras,4410
5,Nómina (Mensual),México,104
6,Nómina (Mensual),Nicaragua,33
7,Nómina (Mensual),Panamá,9
8,Nómina (Mensual),República Dominicana,10
9,Planilla (Semanal),Costa Rica,2329


In [222]:
data['Tipo de Nómina'].unique()

array(['Planilla (Semanal)', 'Nómina (Mensual)', 'Quincenal', nan,
       'Planilla General (Quincenal)', 'Quincenal Confidencial',
       'Planilla Ejecutiva (Mensual)', 'Servicios Profesionales'],
      dtype=object)

In [227]:
data['Tipo de Nómina'].unique().tolist()

['Planilla (Semanal)',
 'Nómina (Mensual)',
 'Quincenal',
 nan,
 'Planilla General (Quincenal)',
 'Quincenal Confidencial',
 'Planilla Ejecutiva (Mensual)',
 'Servicios Profesionales']

In [231]:
data.loc[data['Vacante'] == False]['Usuario'].count()

31172

In [274]:
cpu = data.groupby('Ubicación')\
    .count()\
    .reset_index()\
    [['Ubicación', 'Usuario']]
cpu

,Ubicación,Usuario
0,Barbados,10
1,Costa Rica,5220
2,El Salvador,18639
3,Estados Unidos de América,113
4,Guatemala,33007
5,Honduras,4675
6,México,634
7,Nicaragua,187
8,Panamá,9
9,República Dominicana,2627


In [275]:
source = ColumnDataSource(data=cpu)
cpup = DataTable(
    source=source,
    columns=[
        TableColumn(field="Ubicación", title="Ubicación"),
        TableColumn(field="Usuario", title="Cantidad Contratados"),
    ],
    width=300,
)

In [276]:
show(cpup)

In [285]:
tc = data.loc[data['Vacante'] == False]['Usuario'].count()
stats = PreText(text=f'Total de contratados: {tc}', width=500)
show(stats)

In [330]:
cpd = data.groupby('Nombre Departamento')\
    .count()\
    .reset_index()\
    .sort_values('Usuario', ascending=False)\
    .head(20)\
    [['Nombre Departamento', 'Usuario']]
cpd['angle'] = cpd['Usuario']/cpd['Usuario'].sum() * 2*pi
cpd['color'] = Category20c[len(cpd)]
cpd['Departamento'] = cpd['Nombre Departamento']

cpdp = figure(plot_height=500, title="20 Departamentos con mas empleados", tools="hover", tooltips="@Departamento: @Usuario", x_range=(-0.5, 1.0))

cpdp.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Departamento', source=cpd)
show(cpdp)

In [314]:
cpd

,Nombre Departamento,Usuario,angle,color,Departamento
0,010101-Dirección de Finanzas,1,0.314159,#3182bd,010101-Dirección de Finanzas
1335,San Francisco De Heredia,1,0.314159,#6baed6,San Francisco De Heredia
1325,Rio Claro 194,1,0.314159,#9ecae1,Rio Claro 194
194,Café Barista 118,1,0.314159,#c6dbef,Café Barista 118
1304,Redes y Comunicación,1,0.314159,#e6550d,Redes y Comunicación
1301,Reclutamiento y Seleccion - Compartido,1,0.314159,#fd8d3c,Reclutamiento y Seleccion - Compartido
1293,Puriscal,1,0.314159,#fdae6b,Puriscal
1285,Procesamiento/Producción/Tercerización,1,0.314159,#fdd0a2,Procesamiento/Producción/Tercerización
1283,Procesamiento Posterior,1,0.314159,#31a354,Procesamiento Posterior
1278,Premezla y Microingredientes,1,0.314159,#74c476,Premezla y Microingredientes


In [327]:
cpd = data.groupby('Nombre Departamento')\
    .count()\
    .reset_index()\
    .sort_values('Usuario', ascending=False)\
    .head(20)\
    [['Nombre Departamento', 'Usuario']]

In [328]:
cpd

,Nombre Departamento,Usuario
528,Engorde,1994
526,Empaque,1512
306,Cámara Fría,1359
1275,Postura,1196
704,Operaciones Pollo Campero / Telepizza CAD,1182
1475,Área Fría,1156
1474,Área Caliente,1030
661,Manufactura,944
1441,Ventas,918
1286,Proceso Ulterior,869
